In [157]:
!wget https://github.com/kevin900804/Operation_Research/raw/main/data.xlsx

--2022-08-10 07:22:48--  https://github.com/kevin900804/Operation_Research/raw/main/data.xlsx
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kevin900804/Operation_Research/main/data.xlsx [following]
--2022-08-10 07:22:48--  https://raw.githubusercontent.com/kevin900804/Operation_Research/main/data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13242 (13K) [application/octet-stream]
Saving to: ‘data.xlsx’

data.xlsx           100%[===================>]  12.93K  --.-KB/s    in 0.001s  

2022-08-10 07:22:48 (22.2 MB/s) - ‘data.xlsx’ saved [13242/13242]



In [158]:
import pandas as pd
problem1 = pd.read_excel("/content/data.xlsx", "Problem 1")
problem1.drop(columns = problem1.columns[3:6], inplace = True)

machines = range(1, 4)
jobs = problem1["Job"]
processingTime = problem1["Processing time"]
conflictingJobs = problem1["Conflicting jobs"].apply(lambda x: list(map(int, x.split(','))) if x!= "None" else [])

In [159]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com, https://us-python.pkg.dev/colab-wheels/public/simple/


In [160]:
from gurobipy import *

In [161]:
makespan = Model("makespan")

x = {}
for machine in machines:
  for job in jobs:
      x[machine, job] = makespan.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(machine) + str (job))
z = makespan.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "object value")

makespan.setObjective(z, GRB.MINIMIZE)

makespan.addConstrs(z >= quicksum(processingTime[job - 1] * x[machine, job] for job in jobs) for machine in machines)
makespan.addConstrs(quicksum(x[machine, job] for machine in machines) == 1 for job in jobs)
makespan.addConstrs((quicksum(x[machine, i] for i in conflictingJobs[job - 1]) + x[machine, job]) <= 1 for machine in machines for job in jobs)

makespan.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 63 rows, 46 columns and 174 nonzeros
Model fingerprint: 0xfb40d5b1
Variable types: 1 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 72.0000000
Presolve removed 33 rows and 0 columns
Presolve time: 0.00s
Presolved: 30 rows, 46 columns, 120 nonzeros
Variable types: 0 continuous, 46 integer (45 binary)

Root relaxation: objective 4.266667e+01, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   42.66667    0    4   72.00000   42.66667  40.7%     -    0s
H    0     0                      46.0000000 

In [162]:
print("Result:")
for machine in machines:
  calculateTime= []
  totalTime = 0
  print("Jobs on machine" + str(machine) + ":", end=" ")
  for job in jobs:
    if x[machine, job].x == 1:
      print(str(job) + " ", end = "")
      calculateTime.append(job)
  for i in calculateTime:
    totalTime += processingTime[i-1]
  print("total processing time = " + str(totalTime))

print("The minimized makespan =",makespan.objVal)

Result:
Jobs on machine1: 1 4 6 8 10 total processing time = 42
Jobs on machine2: 3 5 12 14 15 total processing time = 43
Jobs on machine3: 2 7 9 11 13 total processing time = 43
The minimized makespan = 43.0
